## Regex pattern to match declarations of age or year or birth

In [97]:
import re

AGE_CHAR = [
    'novantanove',
    'novantotto',
    'novantasette',
    'novantasei',
    'novantacinque',
    'novantaquattro',
    'novantatre',
    'novantadue',
    'novantuno',
    'novanta',
    'ottantanove',
    'ottantotto',
    'ottantasette',
    'ottantasei',
    'ottantacinque',
    'ottantaquattro',
    'ottantatre',
    'ottantadue',
    'ottantuno',
    'ottanta',
    'settantanove',
    'settantotto',
    'settantasette',
    'settantasei',
    'settantacinque',
    'settantaquattro',
    'settantatre',
    'settantadue',
    'settantuno',
    'settanta',
    'sessantanove',
    'sessantotto',
    'sessantasette',
    'sessantasei',
    'sessantacinque',
    'sessantaquattro',
    'sessantatre',
    'sessantadue',
    'sessantuno',
    'sessanta',
    'cinquantanove',
    'cinquantotto',
    'cinquantasette',
    'cinquantasei',
    'cinquantacinque',
    'cinquantaquattro',
    'cinquantatre',
    'cinquantadue',
    'cinquantuno',
    'cinquanta',
    'quarantanove',
    'quarantotto',
    'quarantasette',
    'quarantasei',
    'quarantacinque',
    'quarantaquattro',
    'quarantatre',
    'quarantadue',
    'quarantuno',
    'quaranta',
    'trentanove',
    'trentotto',
    'trentasette',
    'trentasei',
    'trentacinque',
    'trentaquattro',
    'trentatre',
    'trentadue',
    'trentuno',
    'trenta',
    'ventinove',
    'ventotto',
    'ventisette',
    'ventisei',
    'venticinque',
    'ventiquattro',
    'ventitre',
    'ventidue',
    'ventuno',
    'venti',
    'diciannove',
    'diciotto',
    'diciassette',
    'sedici',
    'quindici',
    'quattordici',
    'tredici'
 ]

# remove last letter of each years_in_words entry, in order to match both
# the noun ("ventiquattro") and the adjective ("ventiquattrenne")
AGE_CHAR_SUFFIX_LONG = [year[:-1] for year in AGE_CHAR]

# keep only the shortest form as a first filter
AGE_CHAR_SUFFIX_SHORT = [
    "tredic",
    "quattordic",
    "quindic",
    "sedic",
    "diciasset",
    "diciott",
    "diciannov",
    "vent",
    "trent",
    "quarant",
    "cinquant",
    "sessant",
    "settant",
    "ottant",
    "novant",
]

AGE_DIGIT = list(range(99,12,-1))

# List of regex patterns for matching Twitter posts mentioning the age of the user
# The patterns are built using the age expressed in digits (e.g. "22" for 22)
AGE_DIGIT_PATTERNS = [
    # Matches phrases like "ho compiuto 22 anni" (I just turned 22)
    # but not "quando ho compiuto 22 anni" (when I turned 22)
    # nor "ho compiuto 22 anni di/de" (I have 22 years of)
    r"(?<!quando\s)(?<!quando)ho\s*compiuto\s*(\d{2})\s*anni(?! de)(?! di)(?!de)(?!di)",
    r"\bcompio\s*(\d{2})\s*anni(?! de)(?! di)(?!de)(?!di)",
    # Matches phrases like "ho 22 anni" (I am 22 years old)
    # but not "da quando/non ho 22 anni" (since I am / I am not 22 years old)
    # nor "ho 22 anni di/de" (I have 22 years of)
    # nor "se ho 22 anni" (if I am 22 years old)
    r"(?<!quando\s)(?<!quando)(?<!non\s)(?<!non)(?<!se\s)(?<!se)ho\s*(\d{2})\s*anni(?! de)(?! di)(?!de)(?!di)",
    # Matches phrases like "faccio 22 anni" (I am turning 22 years old)
    # but not "faccio 22 anni di/de" (I have 22 years of)
    r"\bfaccio\s*(\d{2})\s*anni(?! de)(?! di)(?!de)(?!di)",
    # Matches phrases like "spengo 22 candeline" (I am blowing 22 candles)
    r"\bspengo\s*(\d{2})\s*candeline",
    # Matches phrases like "il mio 22^ compleanno" (my 22nd birthday)
    r"il\s*mio\s*(\d{2})\^\s*comple(?:anno)?",
    # Matches phrases like "sono un 22enne" (I am a 22-year-old...)
    r"\bsono\s*una?\s*(\d{2})\s*enne",
    # Matches phrases like "i miei 22 anni" (my 22 years)
    # r"\bmiei\s*(\d{2})\s*anni",
]

def return_full_age_char_pattern(age_char):
    """
    Returns a list of regex patterns for matching Twitter posts mentioning the age of the user.
    The patterns are built using the age_char parameter, which is a string containing the
    Italian word for the age of the user (e.g. "ventidue" for 22).
    """
    age_char_patterns = [
            # Matches phrases like "ho compiuto ventidue anni" (I just turned twenty-two)
            # but not "quando ho compiuto ventidue anni" (when I turned twenty-two)
            # nor "ho compiuto ventidue anni di/de" (I have twenty-two years of)
            r"(?<!quando\s)(?<!quando)ho\s*compiuto\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            r"\bcompio\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            # Matches phrases like "ho ventidue anni" (I am twenty-two years old),
            # but not "a quando/non ho ventidue anni" (since I am / I am not twenty-two years old)
            # nor "ho ventidue anni di/de" (I have twenty-two years of)
            # nor "se ho ventidue anni" (if I am twenty-two years old)
            r"(?<!quando\s)(?<!quando)(?<!non\s)(?<!non)(?<!se\s)(?<!se)ho\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            # Matches phrases like "faccio ventidue anni" (I am turning twenty-two years old)
            r"\bfaccio\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            # Matches phrases like "spengo ventidue candeline" (I am blowing twenty-two candles)
            r"\bspengo\s*({})\s*candeline".format(age_char),
            # Matches phrases like "mio ventiduesimo comple/compleanno" (my twenty-second birthday)
            r"il\s*mio\s*{}e?simo\s*comple(?:anno)?".format(age_char),
            # Matches phrases like "sono un ventiduenne" (I am twenty-two-years-old...)
            r"\bsono\s*una?\s*({})\s*e?nne".format(age_char),
            # Matches phrases like "i miei ventidue anni" (my twenty-two years)
            # r"\bmiei\s*({}).*\s*anni".format(age_char),
        ]
    return age_char_patterns

def tweet_user_age(tweet):
    """
    Returns the age of the user who posted the tweet, if the tweet contains a mention of the user's age.
    TODO: the age returned by this function should be compared with the creation date of the tweet.
    """
    if len(tweet) > 0:
        # check if the tweet contains a double digit number, but not in a quoted text
        if re.search(r"\d{2}", tweet):
            if not re.search(r"\".*\d{2}.*\"", tweet) \
                and not re.search(r"\“.*\d{2}.*\”", tweet) \
                and not re.search(r"\«.*\d{2}.*\»", tweet):
                # search for age patterns
                for pattern in AGE_DIGIT_PATTERNS:
                    matches = re.findall(pattern, tweet, flags=re.IGNORECASE)
                    if matches:
                        return {"tweet": tweet, "age": int(matches[0])}

        # check if the tweet contains an age expressed in characters
        if re.search(r"{}".format("|".join(AGE_CHAR_SUFFIX_SHORT)), tweet):
            # check what age is expressed in the tweet and retrieve its index
            matching_age_char = re.findall(r"{}".format("|".join(AGE_CHAR_SUFFIX_LONG)), tweet)[0]
            matching_age_char_index = AGE_CHAR_SUFFIX_LONG.index(matching_age_char)
            # check if the age is not in a quoted text
            if not re.search(r"\".*{}.*\"".format(matching_age_char), tweet) \
                and not re.search(r"\“.*{}.*\”".format(matching_age_char), tweet) \
                and not re.search(r"\«.*{}.*\»".format(matching_age_char), tweet):
                # check if also the full form of the age is present in the tweet
                if re.search(r"{}".format(AGE_CHAR[matching_age_char_index]), tweet):
                    patterns = return_full_age_char_pattern(AGE_CHAR[matching_age_char_index])
                else:
                    patterns = return_full_age_char_pattern(AGE_CHAR_SUFFIX_LONG[matching_age_char_index])
                # search for age statements and retrieve age
                for pattern in patterns:
                    matches = re.findall(pattern, tweet, flags=re.IGNORECASE)
                    if matches:
                        return {"tweet": tweet, "age": int(AGE_DIGIT[matching_age_char_index])}

    return {"tweet": tweet, "age": None}

# Example tweets
tweets_should_match = [
    "ho compiuto 50 anni e non mi importa più niente.",
    "Ieri ho compiuto 60 anni e mi sento freschissimo.",
    "Tra un mese compio 25 anni e non vedo l'ora.",
    "Ormai ho 30 anni, damn...",
    "Finalmente faccio 18 anni!!!",
    "Cosa vuoi che ne sappia di vecchiaia, sono una 28enne...",
    "Spengo 40 candeline domani, che emozione.",
    "Grande fiesta per il mio 22^ comple.",
    "Grande fiesta per il mio 22^ compleanno.",
    "--------------------------------------------------------",
    "Ieri ho compiuto sessanta anni e mi sento freschissimo.",
    "Ieri ho compiuto sessant'anni e mi sento freschissimo.",
    "Tra un mese compio venticinque anni e non vedo l'ora.",
    "Ormai ho trent'anni, damn...",
    "Finalmente faccio diciotto anni!!!",
    "Cosa vuoi che ne sappia di vecchiaia, sono una ventottenne...",
    "Spengo quaranta candeline domani, che emozione.",
    "Grande fiesta per il mio ventiduesimo comple.",
    "Grande fiesta per il mio trentunesimo comple.",
    'Mi sento in diritto di dirlo in quanto sono un ventiduenne.',
]

tweets_should_not_match = [
    '@officialsslazio me faccio 30 anni de galera ma te vengo ammazza co le mani mie VERME BAVOSO',
    'Io ho 42 anni di contributi versati, ma "solo" 61 anni di età!',
    'Per il lavoro ho 30 anni di industria sulle spalle.',
    'Le nuove estetiste cinesi sotto casa mi hanno chiesto se ho 24 anni.',
    'Jovanotti: «Ho 50 anni, per me  è un’età assurda»',
    'cara non ho 13 anni, e vedo che vi viene molto difficile tenere una discussione in maniera pacata',
    '“Ho 17 anni e faccio sesso a pagamento: voglio avere tanti soldi e fare shopping”… https://t.co/9NbPpmqRnG',
    "Da quando ho compiuto 50 anni non mi importa più niente.",
    "Guarda sinceramente vedere un ragazzo ridursi a fare questi commenti da 50enne scapola è triste.",
    'l signor "meno male che ho 49 anni".... Lo schifo.',
    "ricordo ancora i miei 22 anni",
    "Porto i miei 77 anni come un giovincello.",
    "Praticamente ho 80 anni", # impossible to detect without semantinc analysis
    "--------------------------------------------------------",
    "@beatriceletre Fisico da ventenne.\nStai un bijoux🌹😘",
    "Da quando ho cinquant'anni non mi importa più niente.",
    'Perché è il mio compleanno e al mio diciottesimo compleanno è mancato mio papà. Al mio compleanno!',
    '“Ho venti anni e faccio sesso a pagamento: voglio avere tanti soldi e fare shopping”… https://t.co/9NbPpmqRnG',
    "Porto i miei settantasette anni come un giovincello.",
    "ricordo ancora i miei ventidue anni",
]

print("######################## Tweets that should match:")
print()
for tweet in tweets_should_match:
    print(tweet_user_age(tweet))

print("\n######################## Tweets that should not match:")
print()
for tweet in tweets_should_not_match:
    print(tweet_user_age(tweet))



######################## Tweets that should match:

{'tweet': 'ho compiuto 50 anni e non mi importa più niente.', 'age': 50}
{'tweet': 'Ieri ho compiuto 60 anni e mi sento freschissimo.', 'age': 60}
{'tweet': "Tra un mese compio 25 anni e non vedo l'ora.", 'age': 25}
{'tweet': 'Ormai ho 30 anni, damn...', 'age': 30}
{'tweet': 'Finalmente faccio 18 anni!!!', 'age': 18}
{'tweet': 'Cosa vuoi che ne sappia di vecchiaia, sono una 28enne...', 'age': 28}
{'tweet': 'Spengo 40 candeline domani, che emozione.', 'age': 40}
{'tweet': 'Grande fiesta per il mio 22^ comple.', 'age': 22}
{'tweet': 'Grande fiesta per il mio 22^ compleanno.', 'age': 22}
{'tweet': '--------------------------------------------------------', 'age': None}
{'tweet': 'Ieri ho compiuto sessanta anni e mi sento freschissimo.', 'age': 60}
{'tweet': "Ieri ho compiuto sessant'anni e mi sento freschissimo.", 'age': 60}
{'tweet': "Tra un mese compio venticinque anni e non vedo l'ora.", 'age': 25}
{'tweet': "Ormai ho trent'anni, damn